In [1]:
%load_ext tensorboard

In [2]:
import datetime
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from tqdm import tqdm

2024-07-16 03:08:48.774580: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-16 03:08:48.868232: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 03:08:48.868338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 03:08:48.871528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-16 03:08:48.892103: I tensorflow/core/platform/cpu_feature_guar

In [4]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import SGD, Adam

In [5]:
import cv2
import urllib
import requests
import PIL.Image

from bs4 import BeautifulSoup

In [6]:
#downloading ship synset
ship_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n03095699")
ship_soup = BeautifulSoup(ship_page.content, "html.parser")

#downloading bike synset:
bike_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n03792782")
bike_soup = BeautifulSoup(bike_page.content, "html.parser")

ship_soup_str = str(ship_soup)
ship_soup_split_url = ship_soup_str.split("\r\n")

bike_soup_str = str(bike_soup)
bike_soup_split_url = bike_soup_str.split("\r\n")

In [ ]:
os.mkdir('./content')
os.mkdir('./content/train')
os.mkdir('./content/train/ships')
os.mkdir('./content/train/bikes')
os.mkdir('./content/validation')
os.mkdir('./content/validation/ships')
os.mkdir('./content/validation/bikes')

In [12]:
print("le: ", len(bike_soup_split_url))

le:  1


In [8]:
image_rows, image_cols = 32, 32

input_shape = (image_rows, image_cols, 3)

def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

n_training_images = 100

#train images for ship
for progress in tqdm(range(n_training_images)):
    if not ship_soup_split_url[progress] == None:
        try:
            I = url_to_image(ship_soup_split_url[progress])
            if len(I.shape) == 3:
                save_path = "./content/train/ships/img" + str(progress) + ".jpeg"
                cv2.imwrite(save_path, I)
        except:
            None

#train images for bikes
for progress in tqdm(range(n_training_images)):
    if not bike_soup_split_url[progress] == None:
        try:
            I = url_to_image(bike_soup_split_url[progress])
            if len(I.shape) == 3:
                save_path = "./content/train/bikes/img" + str(progress) + ".jpeg"
                cv2.imwrite(save_path, I)
        except:
            None

#validation images for ships
for progress in tqdm(range(50)):
    if not ship_soup_split_url[n_training_images+progress] == None:
        try:
            I = url_to_image(ship_soup_split_url[n_training_images+progress])
            if len(I.shape) == 3:
                save_path = "./content/validation/ships/img" + str(progress) + ".jpeg"
                cv2.imwrite(save_path, I)
        except:
            None


#validation images for bikes
for progress in tqdm(range(50)):
    if not bike_soup_split_url[n_training_images+progress] == None:
        try:
            I = url_to_image(bike_soup_split_url[n_training_images+progress])
            if len(I.shape) == 3:
                save_path = "./content/validation/bikes/img" + str(progress) + ".jpeg"
                cv2.imwrite(save_path, I)
        except:
            None

hisfhihihh:  1


  1%|          | 1/100 [00:00<00:00, 7145.32it/s]


IndexError: list index out of range

In [6]:
#defining model

class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(
            96, 
            kernel_size=(11, 11), 
            strides=4, 
            padding="valid", 
            activation="relu", 
            input_shape=input_shape,
            kernel_initializer="he_normal"
        ))
        
        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides = (2, 2),
            padding = "valid",
            data_format = None
        ))

        self.add(Conv2D(
            256,
            kernel_size=(5, 5),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides = (2, 2),
            padding="valid",
            data_format=None
        ))

        self.add(Conv2D(
            384,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(Conv2D(
            384,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides=(2, 2),
            padding="valid",
            data_format=None
        ))

        self.add(Flatten())

        self.add(Dense(4096, activation="relu"))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(1000, activation="relu"))
        self.add(Dense(num_classes, activation="softmax"))

        self.compile(
            optimizer=Adam(0.001),
            loss = 'categorical_crossentropy',
            metrics=["accuracy"]
        )


In [7]:
num_classes = 3

model = AlexNet((227, 227, 3), num_classes)

epochs = 10
batch_size = 32

train_dir = "/home/abhinav/Documents/Work/Hobby/Datasets/Imagenet/synsets"
valid_dir = "/home/abhinav/Documents/Work/Hobby/Datasets/Imagenet/valid_synsets"

Image_height = 227
Image_width = 227

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (Image_height, Image_width),
    color_mode="rgb",
    batch_size=batch_size,
    seed=1,
    shuffle=True,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(Image_height, Image_width),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

train_num = train_generator.samples
valid_num = valid_generator.samples

2024-07-16 03:09:24.181623: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-16 03:09:24.277411: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-16 03:09:24.278099: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 3605 images belonging to 3 classes.
Found 264 images belonging to 3 classes.


In [9]:
model_dir = "./myalexnetmodel.h5"

In [24]:
os.mkdir("./logs")
os.mkdir("./logs/fit")

FileExistsError: [Errno 17] File exists: './logs'

In [8]:
log_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%m%Y-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks_list = [tensorboard_callback]

model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=train_num//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_num//batch_size,
    callbacks=callbacks_list,
    verbose=1
)

model.summary()

2024-07-16 03:09:32.527778: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/10


2024-07-16 03:09:35.524208: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-07-16 03:09:35.717960: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-16 03:09:36.997212: I external/local_xla/xla/service/service.cc:168] XLA service 0x709658c09cf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-16 03:09:36.997271: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-07-16 03:09:37.008188: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1721079577.251017   19452 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


112/112 [==============================] - 65s 496ms/step - loss: 3.4223 - accuracy: 0.5558 - val_loss: 0.9590 - val_accuracy: 0.4961
Epoch 2/10
112/112 [==============================] - 54s 485ms/step - loss: 0.6487 - accuracy: 0.7243 - val_loss: 0.5488 - val_accuracy: 0.7852
Epoch 3/10
112/112 [==============================] - 56s 495ms/step - loss: 0.4790 - accuracy: 0.8161 - val_loss: 0.3175 - val_accuracy: 0.8984
Epoch 4/10
112/112 [==============================] - 55s 492ms/step - loss: 0.4209 - accuracy: 0.8343 - val_loss: 0.5191 - val_accuracy: 0.8320
Epoch 5/10
112/112 [==============================] - 55s 494ms/step - loss: 0.3895 - accuracy: 0.8567 - val_loss: 0.4537 - val_accuracy: 0.8477
Epoch 6/10
112/112 [==============================] - 51s 458ms/step - loss: 0.3334 - accuracy: 0.8766 - val_loss: 0.3147 - val_accuracy: 0.9102
Epoch 7/10
112/112 [==============================] - 40s 357ms/step - loss: 0.3085 - accuracy: 0.8830 - val_loss: 0.4650 - val_accuracy: 0.8

NameError: name 'model_dir' is not defined

In [10]:
model.save(model_dir)

/home/abhinav/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
